# Example for unmixing emission plumes
The notebook demonstrates how the `ddeq.dplume.unmix_plumes` function can be used to unmix overlapping plumes.

In [ ]:
import os

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray

import ddeq

CRS = ccrs.UTM(32)
WGS84 = ccrs.PlateCarree()

from ddeq.smartcarb import DOMAIN

# list of point sources
sources = ddeq.misc.read_point_sources()
sources

Read SMARTCARB satellite and wind data:

In [ ]:
filename = os.path.join(ddeq.DATA_PATH, 'Sentinel_7_CO2_2015042311_o1670_l0483.nc')

data = ddeq.smartcarb.read_level2(filename, co2_noise_scenario='medium',
                                  no2_noise_scenario='high')

lvl_filename = os.path.join(ddeq.DATA_PATH, "SMARTCARB_ERA5-lvl-20150423t1100.nc")
sng_filename = os.path.join(ddeq.DATA_PATH, "SMARTCARB_ERA5-sfc-20150423t1100.nc")

winds = ddeq.era5.read(sng_filename, lvl_filename, method="GNFR_A", sources=sources, times=data.time)

Run plume detection using settings that result in overlapping plumes:

In [ ]:
data = ddeq.dplume.detect_plumes(
    data,
    sources.sel(source=['Berlin', 'Janschwalde', 'Schwarze Pumpe', 'Boxberg']),
    variable='NO2',
    variable_std='NO2_std',
    filter_type='gaussian',
    filter_size=0.5,
    q=0.85,
    crs=CRS,
)

Visualize results showing that the plumes from Boxberg and Schwarze Pumpe overlap:

In [ ]:
fig = ddeq.vis.show_level2(
    data,
    'NO2',
    gas='NO2',
    domain=DOMAIN,
    winds=winds,
    do_zoom=True,
    show_clouds=True
)

Unmix plumes and visualize result:

In [ ]:
data = ddeq.dplume.unmix_plumes(data, sources=sources, winds=winds, pixel_size=2e3)

fig = ddeq.vis.show_level2(
    data,
    'NO2',
    gas='NO2',
    domain=DOMAIN,
    winds=winds,
    do_zoom=True,
    show_clouds=True
)